In [1]:
from lib.controller import Controller
from lib.automation import FileDetector
import time

In [2]:
# Commonly changed settings

new_rig_settings = False
datadir = "./tsm_targets/05-31-22" # All files in this directory + subdirectories are loaded
if new_rig_settings:
    datadir = "C:/Turbo-SM/SMDATA/John/06-06-22" # on new rig
n_group_by_trials = 5 # Treats every n selected files as trials to combine into single ZDA file

fd = FileDetector(directory=datadir)
c = Controller(camera_program=4,
              new_rig_settings=new_rig_settings,
              should_auto_launch=False,
              datadir=datadir)

In [3]:
fd.start_file_detection_loop()
recording_no=1
for i in range(60):
    time.sleep(1)
    new_files = fd.get_unprocessed_file_list()
    if len(new_files) > n_group_by_trials:
        new_files.sort()
        n_process = len(new_files) - (len(new_files) % n_group_by_trials)
        c.select_files(selected_filenames=new_files[:n_process],
                   slice_no=1, 
                   location_no=1,
                   recording_no=recording_no, 
                   filename_base="Untitled",
                   filename_start_no=14,
                   filename_end_no=19)
        c.process_files(n_group_by_trials=n_group_by_trials)
        recording_no += len(new_files)
    
fd.stop_file_detection_loop()

NameError: name 'time' is not defined

# 